In [1]:
# Keras Geant3 Events to True table convolutional autoencoder
import sys, os
print(os.path.dirname(sys.executable))

import pickle
import time
import os
from sys import platform

import numpy as np
import matplotlib.pyplot as plt

from event_display import print_tabled_event

from keras.models import Sequential
from keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, UpSampling2D, Cropping2D, Input, Conv2DTranspose, Dropout
from keras.utils import plot_model
import tensorflow as tf
import keras
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
print(tf.version)
#keras.backend.tensorflow_backend.set_session(tf.Session(config=config))


/home/romanov/anaconda3/envs/ai/bin


2022-12-01 09:43:36.063447: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


AttributeError: module 'tensorflow' has no attribute 'ConfigProto'

In [ ]:
import pandas as pd

all_vals_df = pd.read_feather("2022-11-29_cherepaha_SciGlass-4-1-L_13x13_20x20x200mm_1-10GeV_e-pi-_10kev-each.feather")
all_vals_df["e_div_p"] = all_vals_df.de_sum/(all_vals_df.p*1000)
all_vals_df.head()

,p,prt_name,de_sum,towers,e_div_p
0,1.0,e-,745.830705,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.745831
1,1.0,e-,723.557254,"[0.0, 0.0, 0.33505856326162586, 0.0, 0.0, 0.0,...",0.723557
2,1.0,e-,810.117192,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.810117
3,1.0,e-,853.066921,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.853067
4,1.0,e-,879.492094,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.879492


In [ ]:
towers = np.stack(all_vals_df.towers.to_numpy())
np.shape(towers)
e_answers = np.where(all_vals_df.prt_name == "e-", 1, 0)
pi_answers = np.where(all_vals_df.prt_name == "pi-", 1, 0)
answers = np.column_stack((e_answers, pi_answers))

In [ ]:
inputs = towers

print(f"Inputs shape original = {np.shape(inputs)}")
print(f"answers shape original = {np.shape(answers)}")
print(f"max hit value = {np.max(inputs)}")
# print(f"max e = {np.max(true_e)}")


inputs = np.reshape(inputs, (len(inputs), 13, 13, 1))  # -1 => autodetermine
#answers = np.reshape(answers, (len(answers), 1))  # -1 => autodetermine
# # Pad with 1 row and column of zeroes, so it divides by 2
#inputs = np.pad(inputs, ((0,0), (0,1), (0,1), (0,0)), mode='constant', constant_values=0)
#answers = np.pad(answers, ((0,0), (0,1), (0,1), (0,0)), mode='constant', constant_values=0)
print(f"Inputs shape new = {np.shape(inputs)}")
print(f"Answers shape new = {np.shape(answers)}")


Inputs shape original = (200000, 169)
answers shape original = (200000, 2)
max hit value = 5814.889844006649
Inputs shape new = (200000, 13, 13, 1)
Answers shape new = (200000, 2)


In [ ]:

# print_tabled_event(inputs[0])
# print(answers[0])
# #print_tabled_event(answers[0]*11)
# #print("-----------------------------------")
# #print_tabled_event(inputs[1]*11)
# #print_tabled_event(answers[1]*11)



In [ ]:
model = Sequential()
model.add(Input(shape=(13, 13, 1)))
model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', kernel_initializer='he_normal'))
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', kernel_initializer='he_normal'))
model.add(Conv2D(16, kernel_size=(2, 2), activation='relu', kernel_initializer='he_normal'))
model.add(Conv2D(6, kernel_size=(2, 2), activation='relu', kernel_initializer='he_normal'))
#model.add(Dropout(0.1))
#model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Flatten())
## 128 -> 216
## 64  -> 294
model.add(Dense(294, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))
# model.add(Dense(20, activation='relu'))
# model.add(Dense(274, activation='relu'))
# model.add(Dense(264, activation='relu'))
# model.add(Dense(254, activation='relu'))
# model.add(Dense(242, activation='relu'))
# model.add(Dense(222, activation='relu'))
# model.add(Dense(202, activation='relu'))
# model.add(Dense(182, activation='relu'))
# model.add(Dense(162, activation='relu'))
# model.add(Dense(142, activation='relu'))
# model.add(Dense(121, activation='relu'))
#model.add(Conv2D(1, kernel_size=(2, 2), activation='sigmoid', padding='same'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 12, 12, 64)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        8224      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 16)        2064      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 6)           390       
_________________________________________________________________
flatten (Flatten)            (None, 486)               0         
_________________________________________________________________
dense (Dense)                (None, 294)               143178    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               2

2022-12-01 01:47:32.371415: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-01 01:47:32.372051: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-12-01 01:47:32.395210: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-01 01:47:32.395244: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: romanov-bbox
2022-12-01 01:47:32.395249: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: romanov-bbox
2022-12-01 01:47:32.395349: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.85.2
2022-12-01 01:47:32.395367: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.85.2
2022-12-01 01:47:32.395372: I tensorflow/stream_execu

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc', 'mse', 'mae'])
# output layer
#model.compile(optimizer='adam', loss='mean_squared_error', metrics=['acc', 'mse', 'mae'])
#model.compile(optimizer= 'adam', loss = 'binary_crossentropy')
#model.fit(inputs, answers, epochs=25, batch_size=32, validation_split=0.2)

In [ ]:

history = model.fit(inputs, answers, epochs=25, batch_size=32, validation_split=0.2)

Epoch 1/25


2022-12-01 01:47:32.600055: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-12-01 01:47:32.619216: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3492910000 Hz


5000/5000 [==============================] - 34s 7ms/step - loss: 0.3643 - acc: 0.9608 - mse: 0.0348 - mae: 0.0414 - val_loss: 0.0469 - val_acc: 0.9861 - val_mse: 0.0118 - val_mae: 0.0147
Epoch 2/25
5000/5000 [==============================] - 33s 7ms/step - loss: 0.0445 - acc: 0.9872 - mse: 0.0106 - mae: 0.0145 - val_loss: 0.0461 - val_acc: 0.9852 - val_mse: 0.0130 - val_mae: 0.0157
Epoch 3/25
5000/5000 [==============================] - 31s 6ms/step - loss: 0.0334 - acc: 0.9908 - mse: 0.0079 - mae: 0.0105 - val_loss: 0.0576 - val_acc: 0.9866 - val_mse: 0.0119 - val_mae: 0.0137
Epoch 4/25
5000/5000 [==============================] - 31s 6ms/step - loss: 0.0293 - acc: 0.9914 - mse: 0.0071 - mae: 0.0093 - val_loss: 0.0709 - val_acc: 0.9844 - val_mse: 0.0144 - val_mae: 0.0162
Epoch 5/25
5000/5000 [==============================] - 31s 6ms/step - loss: 0.0253 - acc: 0.9929 - mse: 0.0061 - mae: 0.0079 - val_loss: 0.0251 - val_acc: 0.9925 - val_mse: 0.0064 - val_mae: 0.0077
Epoch 6/25
5000/

In [ ]:
# Set number of CPUS
# config = tf.ConfigProto(device_count={"CPU": 8})
# keras.backend.tensorflow_backend.set_session(tf.Session(config=config))
# history = _
# history.params

{'verbose': 1, 'epochs': 25, 'steps': 5000}

In [ ]:
# Run ML on data

e_min = 0.5
e_max = 1.5
e_1gev_df = all_vals_df.query(f"({e_min} < p < {e_max}) and prt_name=='e-'")
pi_1gev_df = all_vals_df.query(f"({e_min} < p < {e_max}) and prt_name=='pi-'")
e_towers = np.stack(e_1gev_df.towers.to_numpy())
e_inputs = np.reshape(e_towers, (len(e_towers), 13, 13, 1))  # -1 => autodetermine
ml_events_e = model.predict(e_inputs)

In [ ]:
ml_events_e[:,-1:]

NameError: name 'ml_events_e' is not defined

In [ ]:
# Saving the model
model.save(os.path.join('trained_models', "epi-convolution-model.hd5"))

NameError: name 'model' is not defined